# Importance Sampling

사실 Monte Carlo 기법은 어렵지 않습니다. 어려운 부분은 바로 variance입니다. (noise가 되버림) <br>
문제를 해결하기 위해서 할 수 있는 일은 고작 샘플 싸이즈를 늘리는 것이고, 문제를 더욱 악화 시키는 것은, noise를 2배 줄이려면, 2배의 샘플이 아니라 4배의 샘플 싸이즈가 필요로 하게 됩니다. (에러률을 선형으로 떨어뜨리기 위해서는 sample size는 기하급수적으로 늘려야 합니다. ) 

몬테 카를로 기법의 대부분은 사실상 어떻게 하면 variance reduction을 할 것인가에 대한 것입니다. <br>
Importance Sampling 그리고 Quasi Monte Carlo 2개의 기법이 바로 그러한 해결책입니다.  
